# Análisis del riesgo de incumplimiento de los prestatarios

Un banco nos ha solicitado si el estado civíl y el número de hijos de un cliente tienen impacto en el incumplimiento de pago de un préstamo. EL banco nos ha proporcionado los datos sobre la solvencia crediticia de los clientes.


# Inicialización

In [1]:
import pandas as pd

### Objetivo del proyecto:

El informe a continuación tiene como objetivo realizar un análizis en los datos proporcionados para determinar si el número de hijos de los prestamistas afecta con su calificación crediticia o solvencia económica para cancelar a tiempo sus préstamos.

Para llevar a cabo el análisis se plantéa la hipotesis de que el número de hijos si afecta en el cumplimiento de los préstamos emitidos.

# Carga de datos

In [2]:
try:
    df = pd.read_csv('credit_scoring_eng.csv')
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')



# Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo



Realizamos una consulta general para visualizar el estado de nuestros datos.

In [3]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.shape

(21525, 12)

In [5]:
# vamos a mostrar las primeras filas N
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding




Una vez observandas las primeras filas podemos darnos cuenta de 3 inconvenientes:
- La columna 'days_employed' tiene valores negativos.
- La columna 'days_employed' es de tipo de dato float cuando debería ser tipo int.
- Los datos ingresados en la columna 'education' están en mayúsculas y minúsculas, esto crea dificultad al momento de analizarlos.



In [6]:
# Obtener información sobre los datos
print(df['days_employed'][:10])
print()
print(df['days_employed'].dtype)
print()
print(df['education'].unique())

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
5      -926.185831
6     -2879.202052
7      -152.779569
8     -6929.865299
9     -2188.756445
Name: days_employed, dtype: float64

float64

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']




Se detectó que la  misma cantidad de datos ausentes en la columna 'days_employed' es la misma cantidad de datos ausentes que en la columna 'total_income', por lo tanto se puede suponer que estos datos ausentes se debe a clientes desempleados o que todavía no han tenido empleo.

In [7]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
print(df[df['days_employed'].isna()])

       children  days_employed  dob_years            education  education_id  \
12            0            NaN         65  secondary education             1   
26            0            NaN         41  secondary education             1   
29            0            NaN         63  secondary education             1   
41            0            NaN         50  secondary education             1   
55            0            NaN         54  secondary education             1   
...         ...            ...        ...                  ...           ...   
21489         2            NaN         47  Secondary Education             1   
21495         1            NaN         50  secondary education             1   
21497         0            NaN         48    BACHELOR'S DEGREE             0   
21502         1            NaN         42  secondary education             1   
21510         2            NaN         28  secondary education             1   

           family_status  family_status



Para demostrar que nuestra suposición es verdadera o no, vamos a aplicar dos condiciones en nuestra tabla:
- Condición 1: Todos los valores nulos de la columna 'days_employed'
- Condición 2: Todos los valores nulos de la columna 'total_income'

Si la cantidad de filas es igual a la de valores ausentes la suposición es verdadera.

In [66]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
cond_1 = df['days_employed'].isna()
cond_2 = df['total_income'].isna()
print(df[cond_1 & cond_2]['days_employed'].size)
print(df.isna().sum())

2174
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


**Conclusión intermedia**

Ya que el valor de filas coincide con el número de valores ausentes en nuestra tabla filtrada podemos concluir que no existe aleatoriedad en los datos.

A continuación asignaremos un valor a los valores ausentes en base a la media o midiana. Eso lo definiremos de acuerdo a un análisis a continuación.




In [67]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes
porcentaje_val_na = len(df[df['days_employed'].isna()]['days_employed'])/df.shape[0]
print(f'El porcentaje de valores ausentes es de: {int(porcentaje_val_na*100)} %')

El porcentaje de valores ausentes es de: 10 %


## Transformación de datos

In [68]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
print(df['education'].unique())

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


Encontramos dupicados implícitos en la columna 'education'. A contunuación la corregimos. 

In [69]:
# Arregla los registros si es necesario
df['education'] = df['education'].str.lower()


In [70]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido
print(df['education'].unique())


["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


Validamos la columna 'children'.

In [71]:
# Veamos la distribución de los valores en la columna `children`
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Al parecer hubo un problema de tipeo al momento de ingresar los datos, ya que el porcentaje de datos problemáticos es del 0,5% reemplazaremos los datos con -1 por 1 y los datos con 20 por 2. Este cambio no afectará nuestro analisis

In [72]:
# [arregla los datos según tu decisión]
df.loc[df['children'] == -1, 'children'] = 1
df.loc[df['children'] == 20, 'children'] = 2

In [73]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado
df['children'].value_counts()


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


Como observamos en las primeras filas inconsistencias en la columna 'children' vamos a observar la distribución para encontrar datos problemáticos.

In [74]:
# Veamos la distribución de los valores en la columna `days employed`.
df['days_employed'][:10]

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
5      -926.185831
6     -2879.202052
7      -152.779569
8     -6929.865299
9     -2188.756445
Name: days_employed, dtype: float64

In [75]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje
porcentaje = int(df[df['days_employed'] < 0]['days_employed'].count()/df.shape[0]*100)
print(f'El porcentaje de valores problemáticos menores a 0 en la columna days_employed es: {porcentaje}%')

El porcentaje de valores problemáticos menores a 0 en la columna days_employed es: 73%




En este caso debemos primero cambiar la variable 'days_employed' a tipo entero (int) ya que es una variable categórica. Pero primero debemos reemplazar nuestros valores null con 0 para poder transformar nuestro tipo de dato. Luego arreglaremos los valores negativos con la función abs().

In [76]:
# Aborda los valores problemáticos, si existen.
df.loc[df['days_employed'].isna(), 'days_employed'] = 0 # Reemplazamos los valores ausentes por 0
df['days_employed'] = df['days_employed'].astype('int') #Cambiamos el tipo de variable de float a int
df['days_employed'] = df['days_employed'].abs() #Aplicamos el valor absoluto para convertir las filas negativas a enteros positivos



In [77]:
# Comprueba el resultado - asegúrate de que esté arreglado
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,2113,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,3112,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


Ahora realizamos el análisis en la columna 'days_employed'.

In [78]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje
print(df['dob_years'].value_counts().sort_index())# comprobamos la distribución de la columna 'dob_years'
porcentaje_num_pro = (len(df[df['dob_years']==0]['dob_years'])/df.shape[0])*100 #creamos la variable con el porcentaje de números problemáticos
print(porcentaje_num_pro)
print(f'El porcentaje de números problemáticos para la variable \'do_years\': {porcentaje_num_pro}')


0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64
0.4692218350754936
El porcentaje de números problemáticos para la variable 'do_years': 0.4692218350754936



Una vez indentificado los valores problemáticos nos damos cuenta que representan el 0.45%. Para este caso filtraremos la tabla eliminando todos los valores 0.

In [79]:
# Resuelve los problemas en la columna `dob_years`, si existen
df = df[df['dob_years'] != 0]# Filtramos la tabla con los valores diferentes de 0 y persistimos.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21424 non-null  int64  
 1   days_employed     21424 non-null  int64  
 2   dob_years         21424 non-null  int64  
 3   education         21424 non-null  object 
 4   education_id      21424 non-null  int64  
 5   family_status     21424 non-null  object 
 6   family_status_id  21424 non-null  int64  
 7   gender            21424 non-null  object 
 8   income_type       21424 non-null  object 
 9   debt              21424 non-null  int64  
 10  total_income      19260 non-null  float64
 11  purpose           21424 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


In [80]:
# Comprueba el resultado - asegúrate de que esté arreglado
print(df[df['dob_years'] == 0]) # Nos aseguramos que el resultado al filtrar 'dob_years' sea una tabla vacía.

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []




Verificamos los valores problemáticos en la columna 'family_status' con la distribución. Sin embargo para este caso no encontramos valores problemáticos.

In [81]:
# Veamos los valores de la columna
print(df['family_status'].value_counts())# Verificamos si existen valores problemáticos


married              12331
civil partnership     4156
unmarried             2797
divorced              1185
widow / widower        955
Name: family_status, dtype: int64



Verificamos los valores problemáticos en la columna 'gender' con la distribución. Encontramos un valor problemático el cual será eliminado.

In [82]:
# Veamos los valores en la columna
print(df['gender'].value_counts())


F      14164
M       7259
XNA        1
Name: gender, dtype: int64


In [83]:
# Aborda los valores problemáticos, si existen
df.drop(df[df['gender'] == 'XNA'].index, inplace= True) #Eliminamos la fila con el valor problemático igual a 'XNA'



/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [84]:
# Comprueba el resultado - asegúrate de que esté arreglado
print(df['gender'].value_counts())


F    14164
M     7259
Name: gender, dtype: int64



Verificamos los valores problemáticos en la columna 'income_type' con la distribución. No encontramos ningún valor problemático.

In [85]:
# Veamos los valores en la columna
print(df['income_type'].value_counts())

employee                       11064
business                        5064
retiree                         3836
civil servant                   1453
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64



Ya que el porcentaje de duplicados representa el 0.33 % los eliminaremos ya que no afectaría la calidad de nuestro analisis.

In [86]:
# Comprobar los duplicados
porcentaje_duplicados = (df.duplicated().sum()/df.shape[0])* 100
print(f'El porcentaje de duplicados es de: {porcentaje_duplicados} %')

El porcentaje de duplicados es de: 0.33141950240395834 %


In [87]:
# Aborda los duplicados, si existen
df = df.drop_duplicates().reset_index(drop = True)

In [88]:
# Última comprobación para ver si tenemos duplicados
df.duplicated().sum()

0

In [89]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21352 entries, 0 to 21351
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21352 non-null  int64  
 1   days_employed     21352 non-null  int64  
 2   dob_years         21352 non-null  int64  
 3   education         21352 non-null  object 
 4   education_id      21352 non-null  int64  
 5   family_status     21352 non-null  object 
 6   family_status_id  21352 non-null  int64  
 7   gender            21352 non-null  object 
 8   income_type       21352 non-null  object 
 9   debt              21352 non-null  int64  
 10  total_income      19259 non-null  float64
 11  purpose           21352 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB




Hasta el momento nuestro conjunto de datos ha recibido algunos cambios en la etapa de preparación de datos. 
- Se ha reemplazado valores ausentes.
- Se ha cambiado el tipo de datos.
- Se ha validado cada columna de mayor interes la existencia de valores problemáticos.
- Se ha eliminado los duplicados.

Nuestro conjunto de datos procesado tiene 21352 filas, 173 filas menos que nuestro conjunto de datos original. Esto significa 0.8 % menos filas lo cual no es una cantidad significativa para alterar un correcto analisis.


# Trabajar con valores ausentes

Definimos los diccionarios de nuestro conjunto de datos.
- Diccionarion para 'education'.
- Diccionarion pra 'family_status'.

In [90]:
# Encuentra los diccionarios
education = [
    ["bachelor's degree" , 0],
    ['secondary education',1],
    ['some college' , 2],
    ['primary education' , 3],
    ['graduate degree' , 4]
]
columnas = ['name','id']
df_education = pd.DataFrame(education, columns = columnas)
df_education

,name,id
0,bachelor's degree,0
1,secondary education,1
2,some college,2
3,primary education,3
4,graduate degree,4


In [91]:


family = [
    ['married' , 0],
    ['civil partnership',1],
    ['widow/widower' , 2],
    ['divorced' , 3],
    ['unmarried' , 4]
]
columnas = ['name','id']
df_family = pd.DataFrame(family, columns = columnas)
df_family

,name,id
0,married,0
1,civil partnership,1
2,widow/widower,2
3,divorced,3
4,unmarried,4


Conclusión: Una vez identificado los diccoinarios podemos generar tablas a parte con esta información, con el fin de mejorar
nuestro proceso de análisis.

### Restaurar valores ausentes en `total_income`

Para los valores ausentes de de las columnas 'total_income' y days_employed realizaremos un análisis para encontrar la mejor forma de llenar valores ausentes.



In [92]:
# Realizamos una distribaución de la edad para definir las categorías.
df.columns
df['dob_years']
print(df['dob_years'].value_counts().sort_index())

19     14
20     51
21    111
22    183
23    252
24    263
25    357
26    408
27    493
28    503
29    544
30    537
31    559
32    509
33    581
34    601
35    616
36    554
37    536
38    597
39    572
40    607
41    605
42    596
43    512
44    545
45    496
46    472
47    477
48    536
49    508
50    513
51    446
52    484
53    459
54    476
55    443
56    483
57    456
58    454
59    443
60    374
61    354
62    348
63    269
64    260
65    193
66    182
67    167
68     99
69     85
70     65
71     56
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64


In [93]:
# Vamos a escribir una función que calcule la categoría de edad clasificando los valores por decadas.

def calc_edad (row):
    if row < 19:
        return float('Nan')
    
    elif 19 <= row <= 30:
        return '19-30'
    
    elif 31 <= row <= 40:
        return '31-40'
    
    elif 41 <= row <= 50:
        return '41-50'
    
    elif 51 <= row <= 60:
        return '51-60'
    
    elif 61 <= row <= 70:
        return '61-70'
    
    elif row >= 70:
        return '70+'
    
    

In [94]:
# Prueba si la función funciona bien
prue1 = 30
prue2 = 71
prue3 = 18

print(calc_edad(prue1))
print(calc_edad(prue2))
print(calc_edad(prue3))

19-30
70+
nan


In [95]:
# Crear una nueva columna basada en la función

df['age_range'] = df['dob_years'].apply(calc_edad)

df


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_range
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41-50
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,31-40
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,31-40
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,31-40
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,51-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21347,1,4529,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,41-50
21348,0,343937,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,61-70
21349,1,2113,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,31-40
21350,3,3112,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,31-40


In [96]:
# Comprobar cómo los valores en la nueva columna

print(df['age_range'].value_counts())

31-40    5732
41-50    5260
51-60    4518
19-30    3716
61-70    2022
70+       104
Name: age_range, dtype: int64




Se propone que los ingresos dependen de la edad del cliente, vamos a generar una tabla para identificar los ingresos medios y medianas por edad para asignar valores correctamente.

In [97]:
df_valores_completos = df[(df['days_employed'] != 0 ) & (df['total_income'].notna())]
df_valores_completos.info()# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19259 entries, 0 to 21351
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          19259 non-null  int64  
 1   days_employed     19259 non-null  int64  
 2   dob_years         19259 non-null  int64  
 3   education         19259 non-null  object 
 4   education_id      19259 non-null  int64  
 5   family_status     19259 non-null  object 
 6   family_status_id  19259 non-null  int64  
 7   gender            19259 non-null  object 
 8   income_type       19259 non-null  object 
 9   debt              19259 non-null  int64  
 10  total_income      19259 non-null  float64
 11  purpose           19259 non-null  object 
 12  age_range         19259 non-null  object 
dtypes: float64(1), int64(6), object(6)
memory usage: 2.1+ MB


In [98]:
# Examina los valores medios de los ingresos en función de los factores que identificaste

age_grouped = df_valores_completos.groupby('age_range').agg({'total_income':['mean','median']})
age_grouped



total_income            
                   mean      median
age_range                          
19-30      25815.651899  22955.4740
31-40      28376.735148  24825.1865
41-50      28390.207085  24569.9680
51-60      25482.856294  22056.7710
61-70      23245.390243  19705.8550
70+        19575.454327  18611.5935

Conclusión: Hay una brecha considerable entre la media y la mediana, esto indica que hay valores atípicos por tal motivo se tomará en cuenta la mediana.



Una vez realizado una comparación de los ingresos según la edad realizaremos comparaciones de ingresos según nivel de educación.

In [99]:
education_grouped = df_valores_completos.groupby('education').agg({'total_income':['mean','median']})
education_grouped


total_income            
                             mean      median
education                                    
bachelor's degree    33172.428387  28054.5310
graduate degree      27960.024667  25161.5835
primary education    21144.882211  18741.9760
secondary education  24600.353617  21839.4075
some college         29035.057865  25608.7945

Conclusión: Hay una brecha considerable entre la media y la mediana, esto indica que hay valores atípicos por tal motivo se tomará en cuenta la mediana.



Luego de analizar y comparar en mabos factores por edad y por nivel de estudios, se determina que hay una mayor influencia en el nivel de estudios con la cantidad de ingresos. Se tomará la mediana de los ingresos por nivel de estudios para completar los valores ausentes.

In [100]:
df['total_income'] = df['total_income'].fillna(0)
df['total_income'].isna().sum()
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_range
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41-50
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,31-40
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,31-40
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,31-40
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,51-60
5,0,926,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,19-30
6,0,2879,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,41-50
7,0,152,50,secondary education,1,married,0,M,employee,0,21731.829,education,41-50
8,2,6929,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,31-40
9,0,2188,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,41-50


Conclusiones: Se asignó un valor '0' a los valores ausentes, esto nos permitirá realizar una asignación de valores mas precisa con la ayuda de una función.

In [101]:
#df.info()
df.groupby('education').agg({'total_income': ['mean','median']})

total_income            
                             mean      median
education                                    
bachelor's degree    29794.756388  26239.2700
graduate degree      27960.024667  25161.5835
primary education    19570.263323  18195.0890
secondary education  22203.496288  20456.1825
some college         26331.388509  23766.1800

In [102]:
df[df['total_income']== 0]['total_income'].count()

2093

In [103]:
#  Esta función # Recibe como parametros la columna a completar valores ausentes (target_column) 
#y la columna en base(colum to replace) a que rellenamos con la media o mediana.

def fillna_by_other_column(df, target_column, column_to_replace, agg = "median"):
    
    grouped = df.groupby(column_to_replace).agg({target_column: ["mean","median"]})
    for index in grouped.index:
        df.loc[(df[target_column]==0) & (df[column_to_replace] == index),  target_column] = grouped.loc[index,(target_column, agg)]
        
    return df

In [104]:
#Verificamos que los valores de la media según los estudios se hayan ingresado correctamente.
df['total_income'].value_counts()

0.000        2093
31791.384       2
42413.096       2
17312.717       2
28477.825       1
             ... 
10000.392       1
99284.696       1
6264.532        1
27097.085       1
41428.916       1
Name: total_income, Length: 19257, dtype: int64

In [105]:
df = fillna_by_other_column(df, 'total_income','education' )


In [106]:
# Comprueba si tenemos algún error
df[df['total_income'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_range


In [107]:
# Comprobar el número de entradas en las columnas
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21352 entries, 0 to 21351
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21352 non-null  int64  
 1   days_employed     21352 non-null  int64  
 2   dob_years         21352 non-null  int64  
 3   education         21352 non-null  object 
 4   education_id      21352 non-null  int64  
 5   family_status     21352 non-null  object 
 6   family_status_id  21352 non-null  int64  
 7   gender            21352 non-null  object 
 8   income_type       21352 non-null  object 
 9   debt              21352 non-null  int64  
 10  total_income      21352 non-null  float64
 11  purpose           21352 non-null  object 
 12  age_range         21352 non-null  object 
dtypes: float64(1), int64(6), object(6)
memory usage: 2.1+ MB


**Conclusión:** 

La asignación de valores en la columna 'total_income' se realizará en base al nivel de estudio ya que se identificó que había una clara tendencia a mayores ingresos con realación a mayor tiempo de estudios.

###  Restaurar valores en `days_employed`


Para reemplazar los valores ausentes en la columna 'days_employed' usaremos una comparación de la edad de los clientes y definiremos si utilizamos el valor medio o mediana por rango de edad.

In [108]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados

age_grouped_days_employed = df_valores_completos.groupby('age_range').agg({'days_employed':['median','mean']})
age_grouped_days_employed



days_employed               
                 median           mean
age_range                             
19-30            1045.0    2026.553640
31-40            1630.0    4690.621663
41-50            2208.5   16700.800799
51-60            6481.0  153164.611892
61-70          356081.0  288992.207806
70+            360169.5  322556.204082

In [109]:
income_grouped_days_employed_median = df.groupby('income_type')['days_employed'].median()
income_grouped_days_employed_median

income_type
business                         1315
civil servant                    2384
employee                         1366
entrepreneur                      260
paternity / maternity leave      3296
retiree                        360783
student                           578
unemployed                     366413
Name: days_employed, dtype: int64

In [110]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados
age_grouped_days_employed_median = df.groupby('age_range')['days_employed'].median()
age_grouped_days_employed_median

age_range
19-30       938.0
31-40      1416.5
41-50      1872.5
51-60      4518.0
61-70    350356.0
70+      358831.5
Name: days_employed, dtype: float64


Conclusión: ya que tenemos una brecha entre la media y la mediana es evidente que existen valores atípicos por tal motivo usaremos la mediana.

In [111]:
df[df['days_employed']== 0]['days_employed'].count()

2093

In [112]:
# Aplicamos la misma función para llenar datos ausentes de la columna 'total income' pero esta vez para days employed en base
# a al rango de edad.
df = fillna_by_other_column(df, 'days_employed','age_range' )

In [113]:
# Comprueba que la función funciona
df[df['days_employed']== 0]['days_employed'].count()

0

In [114]:
#Verificamos que los valores de la media según los estudios se hayan ingresado correctamente.
df['days_employed'].value_counts()

1416.5      562
1872.5      506
4518.0      470
938.0       355
350356.0    203
           ... 
400056.0      1
347972.0      1
7050.0        1
6975.0        1
2579.0        1
Name: days_employed, Length: 9066, dtype: int64

In [115]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21352 entries, 0 to 21351
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21352 non-null  int64  
 1   days_employed     21352 non-null  float64
 2   dob_years         21352 non-null  int64  
 3   education         21352 non-null  object 
 4   education_id      21352 non-null  int64  
 5   family_status     21352 non-null  object 
 6   family_status_id  21352 non-null  int64  
 7   gender            21352 non-null  object 
 8   income_type       21352 non-null  object 
 9   debt              21352 non-null  int64  
 10  total_income      21352 non-null  float64
 11  purpose           21352 non-null  object 
 12  age_range         21352 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Conclusión: Para este caso utilizamos la edad para asignar los dias de empleo 'days_employed', ya que se identificó una realación directamente proporcional en estas dos variables. A mayor edad mas días de trabajo.

## Clasificación de datos



In [116]:
# Muestra los valores de los datos seleccionados para la clasificación
df['purpose'].sort_values()


7094     building a property
15291    building a property
10294    building a property
19982    building a property
15289    building a property
                ...         
13668       wedding ceremony
10226       wedding ceremony
17461       wedding ceremony
10241       wedding ceremony
9252        wedding ceremony
Name: purpose, Length: 21352, dtype: object

Para la columna 'purpose' tenemos diversos valores pero los cuales se pueden clasificar en categorías.

In [117]:
# Comprobar los valores únicos
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Estos serían los grupos indentificados para clasificar la columna 'purpose'.

- Wedding: Todos los propósitos que tengan relación con esta categoría
- Real State: Todos los clientes que deseen comprar un inmueble
- Car: Todos los clientes que deseen comprar un automobil
- Education: Pago de estudios




In [118]:
# Escribamos una función para clasificar los datos en función de temas comunes

def clas_purpose(row):
    if 'wedding' in row:
        return 'Wedding'
    elif 'hous' in row or 'property' in row or 'real estate' in row:
        return 'Real State'
    elif 'car' in row:
        return 'Car'
    elif 'educa' in row or 'university' in row:
        return 'Education'




In [119]:
# Crea una columna con las categorías y cuenta los valores en ellas
df['purpose_category'] = df['purpose'].apply(clas_purpose)
df['purpose_category'].value_counts()

Real State    10763
Car            4284
Education      3995
Wedding        2310
Name: purpose_category, dtype: int64

También serán clasificados los valores de la columna 'total_income', y se tomará como referencia los cuartiles para clasificar en 4 niveles de ingresos.

In [120]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación
df['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21347    35966.698
21348    24959.969
21349    14347.610
21350    39054.888
21351    13127.587
Name: total_income, Length: 21352, dtype: float64

In [121]:
# Obtener estadísticas resumidas para la columna
df['total_income'].describe()


count     21352.000000
mean      26325.155773
std       15749.700792
min        3306.762000
25%       17223.821250
50%       22585.509500
75%       31321.653000
max      362496.645000
Name: total_income, dtype: float64



Para la clasificación de el nivel de ingreso utilizaremos los siguietes niveles.

- Bajo menos de 17223
- Medio bajo de 17223 a 22585
- Medio alto 22586 a 31321
- Alto mas 31321


In [122]:
# Crear una función para clasificar en diferentes grupos numéricos basándose en rangos
def clas_income(row):
    if row < 17223:
        return 'Bajo'
    
    elif 17223 <= row < 22585:
        return 'Medio Bajo'
    
    elif 22585 <= row < 31321:
        return 'Medio Alto'
    
    elif row >= 31321:
        return 'Alto'
    
   
    
prue1 = 7010
prue2 = 23900
prue3 = 100000

print(clas_income(prue1))
print(clas_income(prue2))
print(clas_income(prue3))


Bajo
Medio Alto
Alto


In [123]:
# Crear una columna con categorías
df['income_level'] = df['total_income'].apply(clas_income)


In [124]:
# Contar los valores de cada categoría para ver la distribución
print(df.groupby('income_level')['total_income'].count())

income_level
Alto          5338
Bajo          5338
Medio Alto    5338
Medio Bajo    5338
Name: total_income, dtype: int64


Conclusión: Al tomar como referencia para la clasificación los cuartiles de los valores de la columna 'total_income', obtuvimos una clasificación distribuida de manera precisa.

## Comprobación de las hipótesis


In [125]:
#Creamos una función que devuelve un data frame con los datos agrupados según la columna ingresada, para contestar las hipótesis planteadas.
def create_inf(df, column):
    resultados = pd.DataFrame()
    resultados['Total'] = df[column].value_counts()
    resultados['Incumplidos'] = df.groupby(column)['debt'].sum()
    resultados['Tasa incumplimiento %'] = (resultados['Incumplidos']/resultados['Total'])*100
    resultados['Tasa incumplimiento %'] = resultados['Tasa incumplimiento %'].map('{:,.2f}'.format)
    
    return resultados

**¿Existe una correlación entre tener hijos y pagar a tiempo?**



In [126]:
# Comprueba los datos sobre los hijos y los pagos puntuales

# Calcular la tasa de incumplimiento en función del número de hijos
create_inf(df, 'children')


,Total,Incumplidos,Tasa incumplimiento %
0,14021,1058,7.55
1,4839,442,9.13
2,2114,202,9.56
3,328,27,8.23
4,41,4,9.76
5,9,0,0.00


**Conclusión**

Existe una relación en la cantidad de hijos, ya que podemos observar que los clientes sin hijos tienen una menor probabilidad de incumplir sus préstamos.


**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [127]:
# Comprueba los datos del estado familiar y los pagos a tiempo

create_inf(df, 'family_status')

# Calcular la tasa de incumplimiento basada en el estado familiar



,Total,Incumplidos,Tasa incumplimiento %
married,12290,927,7.54
civil partnership,4129,386,9.35
unmarried,2794,273,9.77
divorced,1185,85,7.17
widow / widower,954,62,6.50


**Conclusión**

No se presentan ninguna correlación con la situación familiar y el pago de los préstamos ya que tenemos la mayor tasa de incumplimiento para solteros y unión civil, pero una baja tasa de incumplimiento para divorciados.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [128]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo

create_inf(df, 'income_level')

# Calcular la tasa de incumplimiento basada en el nivel de ingresos



,Total,Incumplidos,Tasa incumplimiento %
Medio Bajo,5338,472,8.84
Medio Alto,5338,454,8.51
Alto,5338,382,7.16
Bajo,5338,425,7.96


**Conclusión**

Tampoco se observa una corralción en base al nivel de ingresos. Ya que tenemos las dos mayores tasas para alto y bajo nivel de ingreso pero las menores tasas para los niveles medios. Además la diferencia de tasas entre estos dos grupos es de apenas 1%.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [129]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos
create_inf(df, 'purpose_category')


,Total,Incumplidos,Tasa incumplimiento %
Real State,10763,779,7.24
Car,4284,400,9.34
Education,3995,370,9.26
Wedding,2310,184,7.97


**Conclusión**

Aquí podemos encontrar una correlación bastante clara en base al propósito del crédito ya que para compra de vehículo y educación hay una mayor tasa de incumplimiento.


Hipótesis: La cantidad de hijos de los clientes influye en la tasa de cumplimiento de un préstamo.

Según el análisis realizado podemos dar como verdadera a la hipótesis ya que si se encontró una relación en el incumplimiento de préstamos y la cantidad de hijos.

# Conclusión general 

**Conclusiones del preprocesamiento de datos:**

1. Una vez observandas las primeras filas podemos darnos cuenta de 3 inconvenientes:
    - La columna 'days_employed' tiene valores negativos.
    - La columna 'days_employed' es de tipo de dato float cuando debería ser tipo int.
    - Los datos ingresados en la columna 'education' están en mayúsculas y minúsculas, esto crea dificultad al momento de analizarlos.
    
    
2. Se detectó que la  misma cantidad de datos ausentes en la columna 'days_employed' es la misma cantidad de datos ausentes que en la columna 'total_income', por lo tanto se puede suponer que estos datos ausentes se debe a clientes desempleados o que todavía no han tenido empleo. Por lo tanto la ausencia de estos datos NO ES ALEATORIO.


3. Para los valores problemáticos se realizó las siguientes tareas:
    - Para la columna 'dob_years' se eliminó los valores problemáticos ya que representaban apenas el 0.45% del total.
    - Para tratar la columna 'days_employed' se cambió el tipo de dato a entero y se utilizó el valor absoluto para convertir los valores negativos en positivos. 
    - En la columna género se realizó la eliminación de la única varable diferente de masculino o femenino.
    
4. Para la restauración de datos de las columnas 'total_income' y 'employed_days' se concluye lo siguiente.
    - Al aplicar una función se restauró los datos de una manera mas estandarizada.
    - Para la restauración de 'days_employed' utilizamos la edad para asignar los valores ausentes, ya que se identificó una realación directamente proporcional en estas dos variables. A mayor edad mas días de trabajo.
    - La asignación de valores en la columna 'total_income' se realizó en base al nivel de estudio ya que se identificó que había una clara tendencia a mayores ingresos con realación a mayor tiempo de estudios.


**Conclusiones en referentes a las preguntas planteadas.**

1. Existe una relación en la cantidad de hijos, ya que podemos observar que los clientes sin hijos tienen una menor probabilidad de incumplir sus préstamos.

2. No se presentan ninguna correlación con la situación familiar y el pago de los préstamos ya que tenemos la mayor tasa de incumplimiento para solteros y unión civil, pero una baja tasa de incumplimiento para divorciados.

3. Tampoco se observa una corralción en base al nivel de ingresos. Ya que tenemos las dos mayores tasas para alto y bajo nivel de ingreso pero las menores tasas para los niveles medios. Además la diferencia de tasas entre estos dos grupos es de apenas 1%.

4. Aquí podemos encontrar una correlación bastante clara en base al propósito del crédito ya que para compra de vehículo y educación hay una mayor tasa de incumplimiento.